## **지마켓 상품검색**


* **학습 목표**
    * 검색명 / 페이지 수 / 정렬기준에 따라 크롤링 할 수 있도록 함수를 만들 수 있다.
    * 상품분류, 브랜드명, 상품명, 가격, 배송비, 만족도, 리뷰수, 구매수가 나타나는 데이터프레임을 만들 수 있다.

    ※ <u>각 상품마다 모든 정보가 무조건 있는 것은 아니다. 없는 것도 있기 때문에 예외 처리를 어떻게 해야할 지 고민해야 한다.</u>
<div markdown="1" style="text-align:center; margin-bottom:10px">
<img src="./images/gmarket_html.png" width="50%">
</div> 

## **1) URL 파악하기**

* `https://browse.gmarket.co.kr/search?조건들`
* 조건들: 여러 조건일 때 '&'로 구분한다.

    <table>
        <tr> <th>key</th> <th>설명</th> </tr>
        <tr> <td>keyword</td> <td>검색할 상품 명</td>
        <tr> <td>p</td> <td>페이지 번호</td></tr>
        <tr> 
            <td>s</td> 
            <td> 
                - 1 : 낮은 가격 순<br>
                - 2 : 높은 가격 순<br>
                - 3 : 신규 상품 순<br>
                - 7 : G마켓 링크 순<br>
                - 8 : 판매 인기 순<br>
                - 13 : 상품평 많은 순<br>
            </td> 
        </tr>  
    </table>


## **2) HTML 파악하기**

* div, class='box__information'
    * div, class='box__information-major' 
        * span, class='text__brand'의 text $\rightarrow$ `brand(브랜드명)`
        * span, class='text__item'의 title 속성 $\rightarrow$ `item_name(상품명)`
        * strong, calss='text text__value'의 text $\rightarrow$ `price(가격)`
        * span, class='text__tag' $\rightarrow$ `fee(배송비)`
            * 무료배송인 경우 : img의 alt 속성 
            * 배송비가 있는 경우 : text ('착불'인 경우도 있다)
    * div, class='box__information-score' :
        * span, class='image__awards-points'에서 span의 text $\rightarrow$ `awards(만족도)`<br>
        (output: 만족도 00% 입니다.)
        * li, class='list-item list-item__feedback-count'의 text $\rightarrow$ `feedback(리뷰 수)`<br>
        (output: 상품평(000)건)
        * li, class='list-item list-item__pay-count'의 text $\rightarrow$ `pay_cnt(구매 수)`<br>
        (output: 구매 00건)

## **3) 함수 만들기**

In [118]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup
import re 

# Function
def Gmarket_Search(keyword, page_no, option):
    options = {1: '낮은 가격 순', 2: '높은 가격 순', 3: '신규 상품 순', 7: 'G마켓 링크 순', 8: '판매 인기 순', 13: '상품평 많은 순'}
    columns = ['검색명', '브랜드명', '상품명', '가격', '배송비', '만족도(%)', '리뷰 수', '구매 수']

    # Crawling
    print(f'{keyword}를 {options[option]}으로 검색합니다.')
    print('Start>', end='')
    item_lst = []
    cnt = 0

    ## Each Page
    for p in range(page_no):
        # print message(each page)
        print(f'page {p+1}', end='')

        # HTML 추출
        url = f'https://browse.gmarket.co.kr/search?keyword={keyword}&p={p+1}&s={option}'
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}

        response = req.get(url, headers=headers)
        html = response.text

        # BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')
        # soup.prettify()

        items = soup.select('div.box__item-container')

        ## Each Item
        for i, item in enumerate(items):
            # 브랜드 명
            brand_html = item.select_one('span.text__brand')
            brand = brand_html.text if brand_html!=None else ''
            
            # 상품 명
            item_name = item.select_one('span.text__item')['title']
            
            # 가격
            price = int(re.sub('[^0-9]','',item.select_one('strong.text.text__value').text))
            
            # 배송비
            fee_html = item.select_one('span.text__tag')
            if fee_html.text == '':
                fee = fee_html.img['alt']
            else:
                fee = re.sub('[^0-9]','',fee_html.text)
                fee = fee_html.text if fee=='' else int(fee)

            # 만족도
            award_html = item.select_one('span.image__awards-points > span.for-a11y')
            feedback_html = item.select_one('li.list-item.list-item__feedback-count')
            pay_cnt_html = item.select_one('li.list-item.list-item__pay-count')

            award = int(re.sub('[^0-9]','',award_html.text)) if award_html!=None else ''
            feedback = int(re.sub('[^0-9]','',feedback_html.text)) if feedback_html!=None else ''
            pay_cnt = int(re.sub('[^0-9]','',pay_cnt_html.text)) if pay_cnt_html!=None else ''

            # 행 추가
            cnt += 1
            item_lst.append([keyword, brand, item_name, price, fee, award, feedback, pay_cnt])

            # print message(each item)
            if (i + 1) % 15 == 0:
                print('>', end='')
        
    print('End')
    print(f'{keyword} 상품 {cnt}개의 정보를 수집했습니다')

    item_df = pd.DataFrame(item_lst, columns=columns)
    
    return item_df

## **4) 정보 불러오기**

In [119]:
# Settings
keyword = '이어폰'
page_no = 3
option = 8

gmarket_df = Gmarket_Search(keyword, page_no, option)
gmarket_df

이어폰를 판매 인기 순으로 검색합니다.
Start>page 1>>>>>>page 2>>>>>>page 3>>>>>>End
이어폰 상품 300개의 정보를 수집했습니다


,검색명,브랜드명,상품명,가격,배송비,만족도(%),리뷰 수,구매 수
0,이어폰,젠하이저,EPOS 젠하이저 무선 게임 헤드셋 GSP370,94900,3000,,,108
1,이어폰,삼성전자,갤럭시 버즈2 무선 블루투스 이어폰 SM-R177,111840,무료배송,94,4691,14635
2,이어폰,제이비엘,삼성공식파트너 JBL JR310BT 무선 어린이 블루투스 헤드셋,54900,무료배송,94,258,1299
3,이어폰,젠하이저,(SENNHEISER) (신세계강남점) 랜더스데이 하루특가 젠하이저 인이어 이어폰...,161100,무료배송,94,100,522
4,이어폰,QCY,QCY T13 APP 화이트 TWS 블루투스 이어폰,20800,3000,94,3431,9856
...,...,...,...,...,...,...,...,...
295,이어폰,액센,애플MFi 인증 A161 라이트닝8핀 이어폰 아이폰전용,21800,무료배송,92,75,209
296,이어폰,내셔널지오그래픽,내셔널지오그래픽 에어팟 프로2 세대 러기드 범퍼 락옵션 케이스,19000,무료배송,96,88,190
297,이어폰,QCY,공식정품 QCY 블루투스이어폰 QCY T17APP 파우치포함,18060,2500,86,5,169
298,이어폰,브리츠,블루투스 무선 이어폰 아이폰 갤럭시 호환 핸즈프리 StormTWS7,32900,무료배송,100,1,5
